<a href="https://githubtocolab.com/giswqs/geemap/blob/master/examples/notebooks/cartoee_subplots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install [geemap](https://geemap.org) and [cartopy](https://scitools.org.uk/cartopy/docs/latest/installing.html#installing) if needed. Keep in mind that cartopy can be challenging to install. If you are unable to install cartopy on your computer, you can try Google Colab with this the [notebook example](https://colab.research.google.com/github/giswqs/geemap/blob/master/examples/notebooks/cartoee_colab.ipynb). 

See below the commands to install cartopy and geemap using conda/mamba:

```
conda create -n carto python=3.8
conda activate carto
conda install mamba -c conda-forge
mamba install cartopy scipy -c conda-forge
mamba install geemap -c conda-forge
pip install OWSLib
jupyter notebook
```

## References

- https://github.com/giswqs/geemap/discussions/238
- https://www.net-analysis.com/blog/cartopylayout.html

In [ ]:
# all imports should go here
import pandas as pd
import sys
import os
import subprocess
import datetime
import platform
import datetime
import math

import matplotlib.pyplot as plt

# import seaborn as sb

from geemap import cartoee

import cartopy
import cartopy.crs as ccrs
from cartopy.io.img_tiles import OSM
import cartopy.feature as cfeature
from cartopy.io import shapereader
from cartopy.io.img_tiles import StamenTerrain
from cartopy.io.img_tiles import GoogleTiles
from owslib.wmts import WebMapTileService

from matplotlib.path import Path
import matplotlib.patheffects as PathEffects
from matplotlib import patheffects
import matplotlib.patches as mpatches
import matplotlib.lines as mlines

import numpy as np

import ee
import geemap

geemap.ee_initialize()

# get an image
srtm = ee.Image("CGIAR/SRTM90_V4")
# region = [-180, -60, 180, 85] # define bounding box to request data
region = [110, -45, 160, -10]
vis = {'min': 0, 'max': 3000}  # define visualization parameters for image


fig = plt.figure(figsize=(10, 7))

# ------------------------------- Surrounding frame ------------------------------
# set up frame full height, full width of figure, this must be called first

left = -0.05
bottom = -0.05
width = 1.1
height = 1.05
rect = [left, bottom, width, height]
ax3 = plt.axes(rect)


# turn on the spines we want, ie just the surrounding frame
plt.axis('off')


ax3.text(
    0.01,
    0.01,
    '© Don Cameron, 2017: net-analysis.com. '
    + 'Map generated at '
    + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    fontsize=8,
)


# ---------------------------------  Main Map -------------------------------------
#
# set up main map almost full height (allow room for title), right 80% of figure
BORDERS2_10m = cartopy.feature.NaturalEarthFeature(
    'cultural', 'admin_1_states_provinces', '10m', edgecolor='black', facecolor='none'
)

LAND_10m = cartopy.feature.NaturalEarthFeature(
    'physical',
    'land',
    '10m',
    edgecolor='face',
    facecolor=cartopy.feature.COLORS['land'],
)

RIVERS_10m = cartopy.feature.NaturalEarthFeature(
    'physical',
    'rivers_lake_centerlines',
    '10m',
    edgecolor=cartopy.feature.COLORS['water'],
    facecolor='none',
)

left = 0.2
bottom = 0
width = 0.8
height = 0.90
rect = [left, bottom, width, height]

ax = plt.axes(rect, projection=ccrs.PlateCarree())
# ax.set_extent((150, 155, -30, -23))
# region = [110, -45, 160, -10]
ax.set_extent((110, 160, -45, -10))

cartoee.add_layer(ax, srtm, region=region, vis_params=vis, cmap="terrain")
cartoee.add_colorbar(
    ax, vis, cmap="terrain", loc="bottom", label="Elevation", orientation="horizontal"
)

ax.coastlines(resolution='10m', zorder=2)

# land polygons, including major islands, use cartopy default color
# ax.add_feature(LAND_10m)
# ax.add_feature(RIVERS_10m)
# ax.add_feature(BORDERS2_10m, edgecolor='grey')

# ax.stock_img()
# stock image is good enough for example, but OCEAN_10m could be used, but very slow
#       ax.add_feature(OCEAN_10m)

# ax.gridlines(draw_labels=True, xlocs=[150, 152, 154, 155])
ax.gridlines(draw_labels=True, xlocs=[120, 130, 140, 150])


cartoee.add_scale_bar(ax, length=500, xy=(0.5, 0.08), fontsize=16)
cartoee.add_north_arrow(ax, xy=(0.9, 0.95))


# ---------------------------------Locating Map ------------------------
#
# set up index map 20% height, left 16% of figure
left = 0
bottom = 0
width = 0.16
height = 0.2
rect = [left, bottom, width, height]


ax2 = plt.axes(rect, projection=ccrs.PlateCarree())
ax2.set_extent((-179, 180, -80, 80))
#  ax2.set_global()  will show the whole world as context

ax2.coastlines(resolution='110m', zorder=2)
ax2.add_feature(cfeature.LAND)
ax2.add_feature(cfeature.OCEAN)

ax2.gridlines()


lon0, lon1, lat0, lat1 = ax.get_extent()
box_x = [lon0, lon1, lon1, lon0, lon0]
box_y = [lat0, lat0, lat1, lat1, lat0]

plt.plot(box_x, box_y, color='red', transform=ccrs.Geodetic())


# -------------------------------- Title -----------------------------
# set up map title top 4% of figure, right 80% of figure

left = 0.2
bottom = 0.95
width = 0.8
height = 0.04
rect = [left, bottom, width, height]
ax6 = plt.axes(rect)
ax6.text(0.5, 0.0, 'Multi-Axes Map Example', ha='center', fontsize=20)
plt.axis('off')


# ------------------------------------  Legend -------------------------------------

# legends can be quite long, so set near top of map (0.4 - bottom + 0.5 height = 0.9 - near top)
left = 0
bottom = 0.4
width = 0.16
height = 0.5
rect = [left, bottom, width, height]
rect = [left, bottom, width, height]
ax5 = plt.axes(rect)

# create an array of color patches and associated names for drawing in a legend
# colors are the predefined colors for cartopy features (only for example, Cartopy names are unusual)
colors = sorted(cartopy.feature.COLORS.keys())

# handles is a list of patch handles
handles = []
# names is the list of corresponding labels to appear in the legend
names = []

# for each cartopy defined color, draw a patch, append handle to list, and append color name to names list
for c in colors:
    patch = mpatches.Patch(color=cfeature.COLORS[c], label=c)
    handles.append(patch)
    names.append(c)
# end for

# do some example lines with colors
river = mlines.Line2D(
    [], [], color=cfeature.COLORS['water'], marker='', markersize=15, label='river'
)
coast = mlines.Line2D([], [], color='black', marker='', markersize=15, label='coast')
bdy = mlines.Line2D(
    [], [], color='grey', marker='', markersize=15, label='state boundary'
)
handles.append(river)
handles.append(coast)
handles.append(bdy)
names.append('river')
names.append('coast')
names.append('state boundary')


# create legend
ax5.legend(handles, names)
ax5.set_title('Legend', loc='left')
plt.axis('off')

plt.savefig("test.png", bbox_inches='tight')

plt.show()